# Exploring the Accidents Data Set of the Nairobi Ambulance Challenge

In this notebook we will look further into the Train.csv data set. It contains all accidents with time and location over 18 months between 01/2018 and 07/2019. Check out documentation at https://github.com/caiomiyashiro/geospatial_data_analysis/blob/master/AMLD-2020/Presentation_AMLD_2020.ipynb

## Table of Contents

[Imports and setup](#Imports_setup)<br>
[Extract, transform and load the data](#ETL)<br>
[Data analysis](#Data_analysis)<br>
[Road segment info](#Road_segments)<br>
[Weather data](#Weather)<br>

</br>
</br>
</br>

## Imports and setup <a name="Imports_setup"></a>

### Importing packages

In [1]:
import pandas as pd
import datetime as dt
import math
import seaborn as sns

### Setup

</br>
</br>
</br>

## Extract, transform and load the data <a name="ETL"></a>

### Reading data

In [2]:
df_train = pd.read_csv('../Inputs/Train.csv', parse_dates=['datetime'])
print(df_train.shape)
df_train.head()

(6318, 4)


,uid,datetime,latitude,longitude
0,1,2018-01-01 00:25:46,-1.188850,36.931382
1,2,2018-01-01 02:02:39,-0.662939,37.208730
2,3,2018-01-01 02:31:49,-0.662939,37.208730
3,4,2018-01-01 03:04:01,-1.288087,36.826583
4,5,2018-01-01 03:58:49,-1.188850,36.931382


### Extracting date informations and creating time windows

Extracting date, time, day, weekday, month and year out of the datetime.

In [3]:
df_train["date"] = df_train["datetime"].apply(lambda x: x.date())
df_train["time"] = df_train["datetime"].apply(lambda x: x.time())
df_train["day"] = df_train["datetime"].apply(lambda x: x.day)
df_train["weekday"] = df_train["datetime"].apply(lambda x: x.weekday())

dict_months = {1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
               7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"}
df_train["month"] = df_train["datetime"].apply(lambda x: dict_months.get(x.month))

df_train["year"] = df_train["datetime"].apply(lambda x: x.year)

df_train.head()

,uid,datetime,latitude,longitude,date,time,day,weekday,month,year
0,1,2018-01-01 00:25:46,-1.188850,36.931382,2018-01-01,00:25:46,1,0,Jan,2018
1,2,2018-01-01 02:02:39,-0.662939,37.208730,2018-01-01,02:02:39,1,0,Jan,2018
2,3,2018-01-01 02:31:49,-0.662939,37.208730,2018-01-01,02:31:49,1,0,Jan,2018
3,4,2018-01-01 03:04:01,-1.288087,36.826583,2018-01-01,03:04:01,1,0,Jan,2018
4,5,2018-01-01 03:58:49,-1.188850,36.931382,2018-01-01,03:58:49,1,0,Jan,2018


Creating time windows as demand containers / time window. This helps to interpret demand over time and will fit the submission format.

In [4]:
dict_windows = {1: "00-03", 2: "03-06", 3: "06-09", 4: "09-12", 5: "12-15", 6: "15-18", 7: "18-21", 8: "21-24"}
df_train["time_window"] = df_train["datetime"].apply(lambda x: math.floor(x.hour / 3) + 1)
df_train["time_window"] = df_train["time_window"].apply(lambda x: dict_windows.get(x))
df_train["datetime_window"] = df_train[["date", "time_window"]].apply(lambda x : f"{x[0]}_{x[1]}", axis=1)
df_train.head()

,uid,datetime,latitude,longitude,date,time,day,weekday,month,year,time_window,datetime_window
0,1,2018-01-01 00:25:46,-1.188850,36.931382,2018-01-01,00:25:46,1,0,Jan,2018,00-03,2018-01-01_00-03
1,2,2018-01-01 02:02:39,-0.662939,37.208730,2018-01-01,02:02:39,1,0,Jan,2018,00-03,2018-01-01_00-03
2,3,2018-01-01 02:31:49,-0.662939,37.208730,2018-01-01,02:31:49,1,0,Jan,2018,00-03,2018-01-01_00-03
3,4,2018-01-01 03:04:01,-1.288087,36.826583,2018-01-01,03:04:01,1,0,Jan,2018,03-06,2018-01-01_03-06
4,5,2018-01-01 03:58:49,-1.188850,36.931382,2018-01-01,03:58:49,1,0,Jan,2018,03-06,2018-01-01_03-06


### Creating time windows without accidents

In [5]:
print(df_train["datetime"].min())
print(df_train["datetime"].max())
print(df_train["datetime"].max() - df_train["datetime"].min())

2018-01-01 00:25:46
2019-06-30 20:06:14
545 days 19:40:28


In [6]:
date_list = pd.date_range(df_train["date"].min(), periods=546).tolist()
window_list = ["00-03", "03-06", "06-09", "09-12", "12-15", "15-18", "18-21", "21-24"]
datetime_window_list = []
for day in date_list:
    for time_window in window_list:
        datetime_window_list.append(str(day.date()) + "_" + time_window)

In [23]:
df_complete = pd.DataFrame(datetime_window_list, columns=["datetime_window"])
df_acc_count = df_train.groupby("datetime_window").count()["uid"].reset_index()
df_complete = df_complete.merge(df_acc_count[["datetime_window", "uid"]], on="datetime_window", how='left')

df_complete.rename(columns={"uid":"accidents"}, inplace=True)
df_complete.fillna(value=0, inplace=True)
df_complete["accidents"] = df_complete["accidents"].astype("int")

# New data frame with split value columns 
new = df_complete["datetime_window"].str.split("_", n=1, expand=True) 
# Making separate first name column from new data frame 
df_complete["date"]= new[0] 
# Making separate last name column from new data frame
df_complete["time_window"]= new[1]

df_complete.head(20)

,datetime_window,accidents,date,time_window
0,2018-01-01_00-03,3,2018-01-01,00-03
1,2018-01-01_03-06,4,2018-01-01,03-06
2,2018-01-01_06-09,1,2018-01-01,06-09
3,2018-01-01_09-12,4,2018-01-01,09-12
4,2018-01-01_12-15,1,2018-01-01,12-15
5,2018-01-01_15-18,0,2018-01-01,15-18
6,2018-01-01_18-21,0,2018-01-01,18-21
7,2018-01-01_21-24,0,2018-01-01,21-24
8,2018-01-02_00-03,0,2018-01-02,00-03
9,2018-01-02_03-06,2,2018-01-02,03-06


In [24]:
df_complete.groupby("time_window").sum("accidents")

,accidents
time_window,
00-03,296
03-06,340
06-09,1724
09-12,976
12-15,834
15-18,853
18-21,888
21-24,407


</br>
</br>
</br>

## Data analysis <a name="Data_analysis"></a>

In [ ]:
df_train.groupby("time_window").datetime.count()

### Overall accidents per time window for 2018 and 2019

In [ ]:
fig01 = sns.countplot(data=df_train, x="time_window", palette="Greens")
fig01.set_title("Overall accidents per time window");

### Mean and median accidents per time window for 2018 and 2019

In [ ]:
max_acc = pd.crosstab(df_train["time_window"], df_train["day"]).max(axis=1)
min_acc = pd.crosstab(df_train["time_window"], df_train["day"]).min(axis=1)
mean_acc = pd.crosstab(df_train["time_window"], df_train["day"]).mean(axis=1)
median_acc = pd.crosstab(df_train["time_window"], df_train["day"]).median(axis=1)

In [ ]:
df_stats = pd.DataFrame([max_acc, min_acc, mean_acc, median_acc]).T
df_stats.columns = ["max", "min", "mean", "median"]
df_stats.reset_index(inplace=True)
df_stats.head()

In [ ]:
fig = sns.barplot(data=df_stats, x="time_window", y="max", palette="Reds")
fig.set_title("Maximum amount of accidents per time window");

Note: Minimum cannot be 0 because then we also do not have a column ... need to fix that somehow

In [ ]:
fig = sns.barplot(data=df_stats, x="time_window", y="min", palette="Reds")
fig.set_title("Minimum amount of accidents per time window");

In [ ]:
fig = sns.barplot(data=df_stats, x="time_window", y="mean", palette="Reds")
fig.set_title("Mean of accidents per time window");

In [ ]:
fig = sns.barplot(data=df_stats, x="time_window", y="median", palette="Reds")
fig.set_title("Median amount of accidents per time window");

### Overall accidents per month for 2018

Note: Avoid counting both first halfs (2018 & 2019)

In [ ]:
fig02 = sns.countplot(data=df_train[df_train.year == 2018], x="month", palette="Blues")
fig02.set_title("Overall accidents per month");

### Accidents per weekday

In [ ]:
fig03 = sns.countplot(data=df_train[df_train.year == 2018], x="weekday", palette="Greens")
fig03.set_title("Overall accidents per weekday");

### Time window per day

In [ ]:
sns.catplot(x="time_window", col="weekday",data=df_train[df_train.year == 2018], kind="count", col_wrap=3, palette="Greens");

### Overall accidents per month and time window for 2018

In [ ]:
sns.catplot(x="time_window", col="month",data=df_train[df_train.year == 2018], kind="count", col_wrap=3, palette="Greens");

### Overall accidents per month and day for 2018

In [ ]:
sns.catplot(x="day", col="month",data=df_train[df_train.year == 2018], kind="count", col_wrap=3, palette="Blues");

Note: **It seems that we have missing data for some days.**

In [ ]:
df_check = df_train[(df_train.month == "Sep") & (df_train.day > 20) & (df_train.day < 26)]
df_check

In [ ]:
df_check.shape

</br>
</br>
</br>

## Road segment info <a name="Road_segments"></a>

In [ ]:
df_seg_info = pd.read_csv('../Inputs/Segment_info.csv')
print(df_seg_info.shape)
df_seg_info.head()

</br>
</br>
</br>

## Weather data <a name="Weather"></a>

In [ ]:
df_weather = pd.read_csv('../Inputs/Weather_Nairobi_Daily_GFS.csv', parse_dates=['Date'])
print(df_weather.shape)
df_weather.head()